In [28]:
# https://towardsdatascience.com/turn-your-excel-workbook-into-a-sqlite-database-bc6d4fd206aa
import sqlite3, pandas as pd
org_employees = pd.read_csv(
    'org.csv',
    usecols = ['first_name','last_name','id','parentId'], 
    header=0)

org_employees.head(10)

,first_name,last_name,id,parentId
0,Nathan,Coffield,Coffield-Nathan,Wilson-Katie
1,Pam,Goodwin,Goodwin-Pan,Wilson-Katie
2,Hannah,Muldoon-Davis,Muldoon-Davis-Hannah,Wilson-Katie
3,Blake,Nash,Nash-Blake,Wilson-Katie
4,Kourtney,Spaulding,Spaulding-Kourtney,Wilson-Katie
5,Katie,Wilson,Wilson-Katie,Armstrong
6,Vacant,NaN,Vacant-5,Wilson-Katie
7,Drew,Davis,Davis-Drew,Brownell-Jayne
8,April,Callis,Callis-April,Wood-Dasha
9,Tina,Coyne,Coyne-Tina,Wood-Dasha


In [18]:
db_conn=sqlite3.connect('org.db')
c = db_conn.cursor()
org_employees.columns

Index(['first_name', 'last_name', 'id', 'parentId'], dtype='object')

In [19]:
c.execute(
    """
    CREATE TABLE org_employees (
        first_name TEXT,
        last_name TEXT,
        id TEXT,
        parentId TEXT,
        PRIMARY KEY (id)
    );
    """
)

In [20]:
org_employees.to_sql('org_employees', db_conn, if_exists='append', index=False)

193

In [21]:
qry="""
SELECT first_name, last_name, id, parentId
FROM org_employees
"""
emps=pd.read_sql(qry, db_conn)
emps.head(10)

,first_name,last_name,id,parentId
0,Nathan,Coffield,Coffield-Nathan,Wilson-Katie
1,Pam,Goodwin,Goodwin-Pan,Wilson-Katie
2,Hannah,Muldoon-Davis,Muldoon-Davis-Hannah,Wilson-Katie
3,Blake,Nash,Nash-Blake,Wilson-Katie
4,Kourtney,Spaulding,Spaulding-Kourtney,Wilson-Katie
5,Vacant,None,Vacant-5,Wilson-Katie
6,Katie,Wilson,Wilson-Katie,Armstrong
7,Drew,Davis,Davis-Drew,Brownell-Jayne
8,April,Callis,Callis-April,Wood-Dasha
9,Tina,Coyne,Coyne-Tina,Wood-Dasha


In [25]:
edgelist=pd.merge(emps, emps, left_on='id', right_on='parentId')
edgelist.rename(
  columns={'first_name_x' :'to', 'first_name_y' :'from'},
  inplace=True
)
edgelist=edgelist[['from', 'to']]
edgelist.head(100)

,from,to
0,Nathan,Katie
1,Pam,Katie
2,Hannah,Katie
3,Blake,Katie
4,Kourtney,Katie
...,...,...
95,Jacqlyn,Christina
96,Jacinta,Christina
97,Natalie,Christina
98,Vacant,Christina


In [26]:
# https://towardsdatascience.com/create-organization-diagrams-in-a-few-lines-of-code-the-5-minute-learn-dcca81dac3a2
import networkx as nx
orgchart=nx.from_pandas_edgelist(edgelist, source='to', target='from', create_using=nx.DiGraph)

In [27]:
p=nx.drawing.nx_pydot.to_pydot(orgchart)
p.write_png('orgchart_1.png')

/var/folders/92/xm_ms3bx5bd113b9cmk5zxq40000gn/T/ipykernel_52017/397223888.py:1: DeprecationWarning: nx.nx_pydot.to_pydot depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723
  p=nx.drawing.nx_pydot.to_pydot(orgchart)
